In [15]:
import json
from typing import Dict, List, Tuple, Optional
import re
from difflib import SequenceMatcher

class CNIExtractorKeywordProximity:
    def __init__(self):
        self.label_mappings = {
            'nom': ['NOM', 'SURNAME', 'N0M'],  # N0M pour gérer erreur OCR O/0
            'prenom': ['PRENOMS', 'PRENOM', 'GIVEN NAMES', 'GIVEN NAME'],
            'date_naissance': ['DATE DE NAISSANCE', 'DATE OF BIRTH', 'DATEDE NAISSANCE'],
            'lieu_naissance': ['LIEU DE NAISSANCE', 'PLACE OF BIRTH', 'LIEUDE NAISSANCE'],
            'sexe': ['SEXE', 'SEX'],
            'taille': ['TAILLE', 'HEIGHT', 'SIZE']
        }
        
    def calculate_center(self, polygon: List[List[int]]) -> Tuple[float, float]:
        """Calcule le centre d'un polygone."""
        x_coords = [p[0] for p in polygon]
        y_coords = [p[1] for p in polygon]
        return (sum(x_coords) / len(x_coords), sum(y_coords) / len(y_coords))
    
    def calculate_bbox(self, polygon: List[List[int]]) -> Tuple[int, int, int, int]:
        """Calcule la bounding box d'un polygone."""
        x_coords = [p[0] for p in polygon]
        y_coords = [p[1] for p in polygon]
        return min(x_coords), min(y_coords), max(x_coords), max(y_coords)
    
    def similarity_score(self, str1: str, str2: str) -> float:
        """Calcule un score de similarité entre deux chaînes."""
        return SequenceMatcher(None, str1.upper(), str2.upper()).ratio()
    
    def find_label_indices(self, texts: List[str]) -> Dict[str, List[int]]:
        """Identifie les indices des labels dans le texte OCR."""
        label_indices = {field: [] for field in self.label_mappings}
        
        for idx, text in enumerate(texts):
            text_upper = text.upper().strip()
            
            for field, patterns in self.label_mappings.items():
                for pattern in patterns:
                    # Recherche exacte ou avec similarité élevée
                    if pattern in text_upper or self.similarity_score(text_upper, pattern) > 0.85:
                        label_indices[field].append(idx)
                        break
        
        return label_indices
    
    def find_value_by_proximity(self, label_idx: int, polygons: List, texts: List[str], 
                               scores: List[float], field_type: str) -> Optional[str]:
        """Trouve la valeur associée à un label par proximité spatiale."""
        if label_idx >= len(polygons):
            return None
            
        label_center = self.calculate_center(polygons[label_idx])
        label_bbox = self.calculate_bbox(polygons[label_idx])
        label_height = label_bbox[3] - label_bbox[1]
        label_width = label_bbox[2] - label_bbox[0]
        
        candidates = []
        
        for idx, (polygon, text, score) in enumerate(zip(polygons, texts, scores)):
            if idx == label_idx or not text.strip():
                continue
                
            value_center = self.calculate_center(polygon)
            value_bbox = self.calculate_bbox(polygon)
            
            # Calcul des distances
            dx = value_center[0] - label_center[0]
            dy = value_center[1] - label_center[1]
            
            # Zones de recherche prioritaires
            is_right = dx > 0 and abs(dy) < label_height * 1.5
            is_below = dy > 0 and dy < label_height * 3 and abs(dx) < label_width
            
            if is_right or is_below:
                # Score basé sur la proximité et la confiance OCR
                distance = (dx**2 + dy**2) ** 0.5
                proximity_score = 1 / (1 + distance / 100)  # Normalisation
                combined_score = proximity_score * score
                
                # Priorité aux éléments à droite sur la même ligne
                if is_right and abs(dy) < label_height * 0.5:
                    combined_score *= 1.5
                
                candidates.append({
                    'text': text,
                    'score': combined_score,
                    'distance': distance,
                    'position': 'right' if is_right else 'below'
                })
        
        if not candidates:
            return None
            
        # Sélection du meilleur candidat avec validation
        candidates.sort(key=lambda x: x['score'], reverse=True)
        
        for candidate in candidates:
            value = candidate['text'].strip()
            
            # Validation selon le type de champ
            if self.validate_field(value, field_type):
                return value
                
        return candidates[0]['text'] if candidates else None
    
    def validate_field(self, value: str, field_type: str) -> bool:
        """Valide une valeur selon le type de champ."""
        if not value:
            return False
            
        if field_type == 'date_naissance':
            # Format JJ.MM.AAAA ou JJ/MM/AAAA
            date_pattern = r'^\d{1,2}[./]\d{1,2}[./]\d{4}$'
            return bool(re.match(date_pattern, value))
            
        elif field_type == 'sexe':
            return value.upper() in ['M', 'F', 'MASCULIN', 'FEMININ', 'MALE', 'FEMALE']
            
        elif field_type == 'taille':
            # Format X,XX ou X.XX
            height_pattern = r'^[12][,.]?\d{2}$'
            if re.match(height_pattern, value):
                # Vérifier que la valeur est raisonnable
                height_val = float(value.replace(',', '.'))
                return 1.0 <= height_val <= 2.5
            return False
            
        elif field_type in ['nom', 'prenom']:
            # Au moins 2 caractères alphabétiques
            return len(value) >= 2 and any(c.isalpha() for c in value)
            
        elif field_type == 'lieu_naissance':
            # Au moins 2 caractères
            return len(value) >= 2
            
        return True
    
    def extract(self, ocr_data: Dict) -> Dict[str, Optional[str]]:
        """Extrait les informations de la CNI."""
        texts = ocr_data['rec_texts']
        polygons = ocr_data['rec_polys']
        scores = ocr_data['rec_scores']
        
        # Trouver les indices des labels
        label_indices = self.find_label_indices(texts)
        
        # Extraire les valeurs
        extracted = {}
        
        for field, indices in label_indices.items():
            if indices:
                # Prendre le label avec le meilleur score OCR
                best_idx = max(indices, key=lambda i: scores[i] if i < len(scores) else 0)
                value = self.find_value_by_proximity(best_idx, polygons, texts, scores, field)
                extracted[field] = value
            else:
                extracted[field] = None
        
        # Post-traitement
        extracted = self.post_process(extracted, texts, scores)
        
        return extracted
    
    def post_process(self, extracted: Dict, texts: List[str], scores: List[float]) -> Dict:
        """Post-traitement pour améliorer les résultats."""
        # Si le sexe n'est pas trouvé, chercher 'M' ou 'F' isolé avec bon score
        if not extracted.get('sexe'):
            for text, score in zip(texts, scores):
                if text.strip() in ['M', 'F'] and score > 0.9:
                    extracted['sexe'] = text.strip()
                    break
        
        # Nettoyer la taille
        if extracted.get('taille'):
            taille = extracted['taille']
            # Assurer le format X,XX
            taille = taille.replace('.', ',')
            if ',' not in taille and len(taille) == 3:
                taille = taille[0] + ',' + taille[1:]
            extracted['taille'] = taille
        
        return extracted

# Exemple d'utilisation
if __name__ == "__main__":
    # Charger les données OCR fournies
    ocr_data = json.loads(open("ocr_outputs\ID_Card_Kengali_Fegue_Pacome_1_c48073d1_0\ID_Card_Kengali_Fegue_Pacome_1_res.json").read())
    
    extractor = CNIExtractorKeywordProximity()
    results = extractor.extract(ocr_data)
    
    print("Informations extraites:")
    for field, value in results.items():
        print(f"{field}: {value}")

Informations extraites:
nom: KENGALIFEGUE
prenom: PACOME
date_naissance: 29.06.2004
lieu_naissance: TAILLE/HEIGHT
sexe: M
taille: 1,75


In [18]:
import re
from typing import Dict, List, Optional, Tuple
from datetime import datetime

class CNIExtractorRegexHeuristics:
    def __init__(self):
        # Patterns de détection
        self.patterns = {
            'date': re.compile(r'\b(\d{1,2})[./](\d{1,2})[./](\d{4})\b'),
            'sexe': re.compile(r'^[MF]$'),
            'taille': re.compile(r'\b([12])[,.]?(\d{2})\b'),
            'nom_prenom': re.compile(r'^[A-Z][A-Z\s\-]{2,}$'),
            'lieu': re.compile(r'^[A-Z][A-Z\s\-]{2,}$')
        }
        
        # Mots à exclure (labels OCR)
        self.exclude_words = {
            'NOM', 'SURNAME', 'PRENOMS', 'PRENOM', 'GIVEN', 'NAMES',
            'DATE', 'NAISSANCE', 'BIRTH', 'LIEU', 'PLACE', 'SEXE', 
            'SEX', 'TAILLE', 'HEIGHT', 'PROFESSION', 'OCCUPATION',
            'SIGNATURE', 'REPUBLIQUE', 'CAMEROUN', 'REPUBLIC', 'CAMEROON',
            'NATIONALE', 'NATIONAL', 'IDENTITE', 'IDENTITY', 'CARTE', 'CARD',
            'DE', 'OF', 'DU'
        }
    
    def calculate_position_score(self, polygon: List[List[int]], 
                                image_height: int = 1640) -> float:
        """Calcule un score basé sur la position verticale dans l'image."""
        y_coords = [p[1] for p in polygon]
        y_mean = sum(y_coords) / len(y_coords)
        # Score plus élevé pour les éléments en haut du document
        return 1.0 - (y_mean / image_height)
    
    def extract_date(self, texts: List[str], scores: List[float], 
                    polygons: List) -> Tuple[Optional[str], Optional[int]]:
        """Extrait la date de naissance."""
        current_year = datetime.now().year
        candidates = []
        
        for idx, (text, score) in enumerate(zip(texts, scores)):
            match = self.patterns['date'].search(text)
            if match:
                jour, mois, annee = match.groups()
                annee = int(annee)
                
                # Validation de l'année
                if 1900 <= annee <= current_year - 16:
                    position_score = self.calculate_position_score(polygons[idx])
                    combined_score = score * position_score
                    candidates.append({
                        'value': match.group(),
                        'index': idx,
                        'score': combined_score,
                        'year': annee
                    })
        
        if candidates:
            # Préférer les dates avec années raisonnables (20-80 ans)
            best = max(candidates, key=lambda x: x['score'])
            return best['value'], best['index']
        
        return None, None
    
    def extract_sexe(self, texts: List[str], scores: List[float]) -> Optional[str]:
        """Extrait le sexe."""
        for text, score in zip(texts, scores):
            text = text.strip()
            if self.patterns['sexe'].match(text) and score > 0.9:
                return text
        return None
    
    def extract_taille(self, texts: List[str], scores: List[float]) -> Optional[str]:
        """Extrait la taille."""
        for text, score in zip(texts, scores):
            match = self.patterns['taille'].search(text)
            if match and score > 0.8:
                metres, centimetres = match.groups()
                # Reconstituer au format X,XX
                taille_str = f"{metres},{centimetres}"
                taille_val = float(f"{metres}.{centimetres}")
                
                # Validation de la valeur
                if 1.0 <= taille_val <= 2.5:
                    return taille_str
        
        return None
    
    def is_valid_name(self, text: str) -> bool:
        """Vérifie si le texte peut être un nom/prénom."""
        text = text.strip()
        
        # Exclure les mots réservés
        if text.upper() in self.exclude_words:
            return False
        
        # Doit contenir au moins 3 caractères alphabétiques
        if len(text) < 3:
            return False
        
        # Majoritairement alphabétique
        alpha_count = sum(1 for c in text if c.isalpha())
        if alpha_count < len(text) * 0.8:
            return False
        
        # Pattern de nom
        return bool(self.patterns['nom_prenom'].match(text))
    
    def extract_nom_prenom(self, texts: List[str], scores: List[float], 
                          polygons: List, date_idx: Optional[int]) -> Dict[str, Optional[str]]:
        """Extrait le nom et le prénom basé sur les heuristiques de position."""
        candidates = []
        
        # Calculer la hauteur maximale pour le tiers supérieur
        all_y = [p[1] for poly in polygons for p in poly]
        max_y = max(all_y) if all_y else 1640
        threshold_y = max_y * 0.4  # Zone supérieure
        
        for idx, (text, score, polygon) in enumerate(zip(texts, scores, polygons)):
            if not self.is_valid_name(text):
                continue
            
            y_coords = [p[1] for p in polygon]
            y_mean = sum(y_coords) / len(y_coords)
            
            # Doit être dans la partie supérieure et avant la date
            if y_mean < threshold_y and (date_idx is None or idx < date_idx):
                if score > 0.85:
                    candidates.append({
                        'text': text.strip(),
                        'index': idx,
                        'y_position': y_mean,
                        'score': score
                    })
        
        # Trier par position verticale
        candidates.sort(key=lambda x: x['y_position'])
        
        result = {'nom': None, 'prenom': None}
        
        # Les deux premiers candidats valides sont probablement nom et prénom
        if len(candidates) >= 1:
            result['nom'] = candidates[0]['text']
        if len(candidates) >= 2:
            result['prenom'] = candidates[1]['text']
        
        return result
    
    def extract_lieu_naissance(self, texts: List[str], scores: List[float], 
                              polygons: List, date_idx: Optional[int]) -> Optional[str]:
        """Extrait le lieu de naissance."""
        if date_idx is None:
            return None
        
        # Chercher dans une fenêtre après la date
        window_start = date_idx + 1
        window_end = min(date_idx + 5, len(texts))
        
        for idx in range(window_start, window_end):
            text = texts[idx].strip()
            score = scores[idx]
            
            # Vérifier que c'est un nom de lieu valide
            if (self.is_valid_name(text) and 
                text.upper() not in self.exclude_words and
                score > 0.85):
                return text
        
        return None
    
    def extract(self, ocr_data: Dict) -> Dict[str, Optional[str]]:
        """Extraction principale utilisant regex et heuristiques."""
        texts = ocr_data['rec_texts']
        scores = ocr_data['rec_scores']
        polygons = ocr_data['rec_polys']
        
        # Extraction séquentielle
        results = {}
        
        # 1. Date (point d'ancrage important)
        date_value, date_idx = self.extract_date(texts, scores, polygons)
        results['date_naissance'] = date_value
        
        # 2. Sexe (indépendant)
        results['sexe'] = self.extract_sexe(texts, scores)
        
        # 3. Taille (indépendant)
        results['taille'] = self.extract_taille(texts, scores)
        
        # 4. Nom et Prénom (basé sur position)
        nom_prenom = self.extract_nom_prenom(texts, scores, polygons, date_idx)
        results.update(nom_prenom)
        
        # 5. Lieu de naissance (basé sur proximité avec date)
        results['lieu_naissance'] = self.extract_lieu_naissance(
            texts, scores, polygons, date_idx
        )
        
        # Post-traitement et validation croisée
        results = self.post_process(results, texts, scores)
        
        return results
    
    def post_process(self, results: Dict, texts: List[str], 
                    scores: List[float]) -> Dict:
        """Post-traitement et corrections finales."""
        
        # Si pas de lieu trouvé, chercher DSCHANG ou autres villes connues
        if not results.get('lieu_naissance'):
            villes_cameroun = ['DOUALA', 'YAOUNDE', 'DSCHANG', 'BAMENDA', 
                              'BAFOUSSAM', 'GAROUA', 'MAROUA', 'NGAOUNDERE']
            for text in texts:
                text_upper = text.strip().upper()
                if text_upper in villes_cameroun:
                    results['lieu_naissance'] = text
                    break
        
        # Validation croisée nom/prénom
        if results.get('nom') and results.get('prenom'):
            # Si le "nom" contient des mots du label, inverser
            if any(word in results['nom'].upper() for word in ['PRENOM', 'GIVEN']):
                results['nom'], results['prenom'] = results['prenom'], results['nom']
        
        # Nettoyer les valeurs
        for key in results:
            if results[key]:
                results[key] = results[key].strip()
        
        return results
    
    def confidence_score(self, results: Dict) -> float:
        """Calcule un score de confiance global."""
        filled_fields = sum(1 for v in results.values() if v is not None)
        total_fields = len(results)
        return filled_fields / total_fields if total_fields > 0 else 0.0

# Exemple d'utilisation
if __name__ == "__main__":
    import json
    
    # Simuler le chargement des données
    ocr_data = json.loads(open("ocr_outputs\ID_Card_Kengali_Fegue_Pacome_1_c48073d1_0\ID_Card_Kengali_Fegue_Pacome_1_res.json").read())

    
    extractor = CNIExtractorRegexHeuristics()
    results = extractor.extract(ocr_data)
    confidence = extractor.confidence_score(results)
    
    print("Résultats extraits:")
    for field, value in results.items():
        print(f"{field}: {value}")
    print(f"\nScore de confiance: {confidence:.2%}")

Résultats extraits:
date_naissance: 29.06.2004
sexe: M
taille: 1,75
nom: REPUBLIQUE DU CAMEROUN
prenom: REPUBLIC OF CAMEROON
lieu_naissance: CARTENATIONALE DIDENTITE

Score de confiance: 100.00%


In [14]:
import numpy as np
from typing import Dict, List, Tuple, Optional
import re

class CNIExtractorTemplateMatching:
    def __init__(self):
        # Template de référence : positions relatives des champs (x, y, largeur, hauteur)
        # Valeurs en pourcentage de la taille totale du document
        self.template_zones = {
            'nom': {
                'x': 0.35, 'y': 0.15, 'width': 0.30, 'height': 0.05,
                'anchors': ['NOM', 'SURNAME']
            },
            'prenom': {
                'x': 0.35, 'y': 0.20, 'width': 0.30, 'height': 0.05,
                'anchors': ['PRENOMS', 'GIVEN NAMES']
            },
            'date_naissance': {
                'x': 0.35, 'y': 0.30, 'width': 0.25, 'height': 0.05,
                'anchors': ['DATE DE NAISSANCE', 'DATE OF BIRTH']
            },
            'lieu_naissance': {
                'x': 0.35, 'y': 0.35, 'width': 0.30, 'height': 0.05,
                'anchors': ['LIEU DE NAISSANCE', 'PLACE OF BIRTH']
            },
            'sexe': {
                'x': 0.35, 'y': 0.40, 'width': 0.10, 'height': 0.05,
                'anchors': ['SEXE', 'SEX']
            },
            'taille': {
                'x': 0.50, 'y': 0.40, 'width': 0.15, 'height': 0.05,
                'anchors': ['TAILLE', 'HEIGHT']
            }
        }
        
        # Points d'ancrage pour l'alignement
        self.alignment_anchors = [
            'REPUBLIQUE DU CAMEROUN',
            'REPUBLIC OF CAMEROON',
            'CARTE NATIONALE',
            'NATIONAL IDENTITY CARD'
        ]
    
    def find_document_bounds(self, polygons: List) -> Tuple[int, int, int, int]:
        """Trouve les limites du document dans l'image."""
        all_points = [point for poly in polygons for point in poly]
        if not all_points:
            return 0, 0, 2000, 1600
        
        x_coords = [p[0] for p in all_points]
        y_coords = [p[1] for p in all_points]
        
        return min(x_coords), min(y_coords), max(x_coords), max(y_coords)
    
    def detect_anchor_points(self, texts: List[str], polygons: List) -> List[Dict]:
        """Détecte les points d'ancrage pour l'alignement."""
        anchors = []
        
        for idx, text in enumerate(texts):
            text_upper = text.strip().upper()
            
            # Vérifier si c'est un point d'ancrage
            for anchor_text in self.alignment_anchors:
                if anchor_text in text_upper or self.similarity(text_upper, anchor_text) > 0.8:
                    # Calculer le centre du polygone
                    polygon = polygons[idx]
                    center_x = sum(p[0] for p in polygon) / len(polygon)
                    center_y = sum(p[1] for p in polygon) / len(polygon)
                    
                    anchors.append({
                        'text': text,
                        'expected': anchor_text,
                        'center': (center_x, center_y),
                        'polygon': polygon,
                        'index': idx
                    })
                    break
        
        return anchors
    
    def similarity(self, str1: str, str2: str) -> float:
        """Calcule la similarité entre deux chaînes."""
        set1 = set(str1.split())
        set2 = set(str2.split())
        intersection = set1.intersection(set2)
        union = set1.union(set2)
        return len(intersection) / len(union) if union else 0.0
    
    def estimate_transformation(self, anchors: List[Dict], 
                               doc_bounds: Tuple[int, int, int, int]) -> Dict:
        """Estime la transformation géométrique du document."""
        min_x, min_y, max_x, max_y = doc_bounds
        doc_width = max_x - min_x
        doc_height = max_y - min_y
        
        # Paramètres de transformation par défaut
        transform = {
            'scale_x': 1.0,
            'scale_y': 1.0,
            'rotation': 0.0,
            'offset_x': min_x,
            'offset_y': min_y,
            'width': doc_width,
            'height': doc_height
        }
        
        # Si on a au moins 2 points d'ancrage, estimer la rotation
        if len(anchors) >= 2:
            # Prendre les deux premiers points d'ancrage
            p1 = anchors[0]['center']
            p2 = anchors[1]['center']
            
            # Calculer l'angle de rotation
            dx = p2[0] - p1[0]
            dy = p2[1] - p1[1]
            
            # Angle en radians (assumant que le document devrait être horizontal)
            angle = np.arctan2(dy, dx)
            
            # Normaliser l'angle (proche de 0 ou 90 degrés)
            if abs(angle) < np.pi/4:
                transform['rotation'] = angle
            elif abs(angle - np.pi/2) < np.pi/4:
                transform['rotation'] = angle - np.pi/2
        
        return transform
    
    def transform_point(self, x: float, y: float, transform: Dict, inverse: bool = False) -> Tuple[float, float]:
        """Applique ou inverse une transformation à un point."""
        if inverse:
            # Transformation inverse : template -> image
            # Dénormaliser
            x_abs = x * transform['width'] + transform['offset_x']
            y_abs = y * transform['height'] + transform['offset_y']
            
            # Appliquer la rotation inverse si nécessaire
            if transform['rotation'] != 0:
                cos_r = np.cos(-transform['rotation'])
                sin_r = np.sin(-transform['rotation'])
                cx = transform['offset_x'] + transform['width'] / 2
                cy = transform['offset_y'] + transform['height'] / 2
                
                x_rot = cos_r * (x_abs - cx) - sin_r * (y_abs - cy) + cx
                y_rot = sin_r * (x_abs - cx) + cos_r * (y_abs - cy) + cy
                return x_rot, y_rot
            
            return x_abs, y_abs
        else:
            # Transformation directe : image -> template
            # Appliquer la rotation si nécessaire
            if transform['rotation'] != 0:
                cos_r = np.cos(transform['rotation'])
                sin_r = np.sin(transform['rotation'])
                cx = transform['offset_x'] + transform['width'] / 2
                cy = transform['offset_y'] + transform['height'] / 2
                
                x_rot = cos_r * (x - cx) - sin_r * (y - cy) + cx
                y_rot = sin_r * (x - cx) + cos_r * (y - cy) + cy
                x, y = x_rot, y_rot
            
            # Normaliser
            x_norm = (x - transform['offset_x']) / transform['width']
            y_norm = (y - transform['offset_y']) / transform['height']
            
            return x_norm, y_norm
    
    def extract_from_zone(self, zone_def: Dict, texts: List[str], 
                         polygons: List, scores: List[float], 
                         transform: Dict) -> Optional[str]:
        """Extrait le texte d'une zone spécifique après transformation."""
        # Convertir la zone template en coordonnées image
        zone_x1, zone_y1 = self.transform_point(
            zone_def['x'], zone_def['y'], transform, inverse=True
        )
        zone_x2, zone_y2 = self.transform_point(
            zone_def['x'] + zone_def['width'], 
            zone_def['y'] + zone_def['height'], 
            transform, inverse=True
        )
        
        # Assurer l'ordre correct des coordonnées
        x1, x2 = min(zone_x1, zone_x2), max(zone_x1, zone_x2)
        y1, y2 = min(zone_y1, zone_y2), max(zone_y1, zone_y2)
        
        # Étendre légèrement la zone pour la tolérance
        margin_x = (x2 - x1) * 0.2
        margin_y = (y2 - y1) * 0.2
        x1 -= margin_x
        x2 += margin_x
        y1 -= margin_y
        y2 += margin_y
        
        candidates = []
        
        for idx, (text, polygon, score) in enumerate(zip(texts, polygons, scores)):
            if not text.strip():
                continue
            
            # Calculer le centre du polygone
            center_x = sum(p[0] for p in polygon) / len(polygon)
            center_y = sum(p[1] for p in polygon) / len(polygon)
            
            # Vérifier si le centre est dans la zone
            if x1 <= center_x <= x2 and y1 <= center_y <= y2:
                # Calculer le score basé sur la proximité au centre de la zone
                zone_center_x = (x1 + x2) / 2
                zone_center_y = (y1 + y2) / 2
                distance = ((center_x - zone_center_x)**2 + (center_y - zone_center_y)**2) ** 0.5
                
                # Normaliser la distance
                max_distance = ((x2 - x1)**2 + (y2 - y1)**2) ** 0.5
                proximity_score = 1 - (distance / max_distance) if max_distance > 0 else 1
                
                # Exclure les mots d'ancrage
                is_anchor = False
                for anchor in zone_def.get('anchors', []):
                    if anchor.upper() in text.upper():
                        is_anchor = True
                        break
                
                if not is_anchor:
                    candidates.append({
                        'text': text.strip(),
                        'score': score * proximity_score,
                        'ocr_score': score,
                        'proximity': proximity_score
                    })
        
        # Sélectionner le meilleur candidat
        if candidates:
            # Trier par score combiné
            candidates.sort(key=lambda x: x['score'], reverse=True)
            return candidates[0]['text']
        
        return None
    
    def validate_extraction(self, field: str, value: Optional[str]) -> bool:
        """Valide la valeur extraite selon le type de champ."""
        if not value:
            return False
        
        if field == 'date_naissance':
            # Format date JJ.MM.AAAA ou JJ/MM/AAAA
            pattern = r'^\d{1,2}[./]\d{1,2}[./]\d{4}'
            return bool(re.match(pattern, value))
        
        elif field == 'sexe':
            return value.upper() in ['M', 'F']
        
        elif field == 'taille':
            # Format taille X,XX ou X.XX
            pattern = r'^[12][,.]?\d{2}'
            return bool(re.match(pattern, value))
        
        elif field in ['nom', 'prenom', 'lieu_naissance']:
            # Au moins 2 caractères alphabétiques
            return len(value) >= 2 and any(c.isalpha() for c in value)
        
        return True
    
    def extract(self, ocr_data: Dict) -> Dict[str, Optional[str]]:
        """Extraction principale par template matching."""
        texts = ocr_data['rec_texts']
        polygons = ocr_data['rec_polys']
        scores = ocr_data['rec_scores']
        
        # 1. Trouver les limites du document
        doc_bounds = self.find_document_bounds(polygons)
        
        # 2. Détecter les points d'ancrage
        anchors = self.detect_anchor_points(texts, polygons)
        
        # 3. Estimer la transformation
        transform = self.estimate_transformation(anchors, doc_bounds)
        
        # 4. Extraire les champs depuis les zones du template
        results = {}
        
        for field, zone_def in self.template_zones.items():
            value = self.extract_from_zone(zone_def, texts, polygons, scores, transform)
            
            # Valider l'extraction
            if self.validate_extraction(field, value):
                results[field] = value
            else:
                results[field] = None
        
        # 5. Post-traitement et récupération des champs manquants
        results = self.fallback_extraction(results, texts, scores)
        
        return results
    
    def fallback_extraction(self, results: Dict, texts: List[str], 
                           scores: List[float]) -> Dict:
        """Méthode de secours pour les champs non trouvés."""
        
        # Si la date n'est pas trouvée, chercher par pattern
        if not results.get('date_naissance'):
            date_pattern = re.compile(r'\d{1,2}[./]\d{1,2}[./]\d{4}')
            for text, score in zip(texts, scores):
                if date_pattern.match(text) and score > 0.9:
                    results['date_naissance'] = text
                    break
        
        # Si le sexe n'est pas trouvé
        if not results.get('sexe'):
            for text, score in zip(texts, scores):
                if text.strip() in ['M', 'F'] and score > 0.9:
                    results['sexe'] = text.strip()
                    break
        
        # Si la taille n'est pas trouvée
        if not results.get('taille'):
            taille_pattern = re.compile(r'[12][,.]?\d{2}')
            for text, score in zip(texts, scores):
                match = taille_pattern.match(text)
                if match and score > 0.9:
                    # Formater correctement
                    taille = text.replace('.', ',')
                    if ',' not in taille and len(taille) == 3:
                        taille = taille[0] + ',' + taille[1:]
                    results['taille'] = taille
                    break
        
        # Nettoyer les valeurs
        for key in results:
            if results[key]:
                results[key] = results[key].strip()
        
        return results
    
    def compute_confidence(self, results: Dict) -> Dict[str, float]:
        """Calcule un score de confiance pour chaque champ extrait."""
        confidence = {}
        for field, value in results.items():
            if value:
                # Confiance basée sur la validation
                if self.validate_extraction(field, value):
                    confidence[field] = 0.9
                else:
                    confidence[field] = 0.5
            else:
                confidence[field] = 0.0
        
        return confidence

# Exemple d'utilisation
if __name__ == "__main__":
    import json
    
    # Charger les données OCR
    ocr_data = json.loads(open("ocr_outputs\ID_Card_Kengali_Fegue_Pacome_1_c48073d1_0\ID_Card_Kengali_Fegue_Pacome_1_res.json").read())

    
    extractor = CNIExtractorTemplateMatching()
    results = extractor.extract(ocr_data)
    confidence = extractor.compute_confidence(results)
    
    print("Résultats extraits par template matching:")
    for field, value in results.items():
        conf = confidence.get(field, 0.0)
        print(f"{field}: {value} (confiance: {conf:.0%})")

Résultats extraits par template matching:
nom: KENGALIFEGUE (confiance: 90%)
prenom: KENGALIFEGUE (confiance: 90%)
date_naissance: 29.06.2004 (confiance: 90%)
lieu_naissance: PACOME (confiance: 90%)
sexe: M (confiance: 90%)
taille: 1,75 (confiance: 90%)


In [39]:
import re
from typing import Dict, List, Tuple, Optional
from difflib import SequenceMatcher

class CNIExtractor18F:
    """
    Extracteur de CNI camerounaise utilisant une approche par élimination
    avec détection floue des ancres.
    """
    
    def __init__(self, quality_threshold: float = 0.5, similarity_threshold: float = 0.70, debug: bool = True):
        """
        Initialise l'extracteur.
        
        Args:
            quality_threshold: Seuil minimal de qualité OCR pour procéder
            similarity_threshold: Seuil de similarité pour la détection des ancres
            debug: Active les logs de débogage
        """
        self.quality_threshold = quality_threshold
        self.similarity_threshold = similarity_threshold
        self.debug = debug
        
        # Ancres possibles pour chaque champ
        self.anchors = {
            'nom': ['NOM', 'SURNAME', 'NOM/SURNAME'],
            'prenom': ['PRENOMS', 'PRENOM', 'GIVEN NAMES', 'GIVEN NAME', 'PRENOMS/GIVEN NAMES'],
            'lieu_naissance': ['LIEU DE NAISSANCE', 'PLACE OF BIRTH', 
                              'LIEU DENAISSANCE', 'PLACEOF BIRTH',
                              'LIEU DE NAISSANCE/PLACEOF BIRTH'],
            'profession': ['PROFESSION', 'OCCUPATION', 'PROFESSION/OCCUPATION']
        }
        
        # Tous les labels possibles (pour filtrage)
        self.all_labels = set()
        for labels in self.anchors.values():
            self.all_labels.update(labels)
        self.all_labels.update(['DATE DE NAISSANCE', 'DATE OF BIRTH', 
                                'SEXE', 'SEX', 'TAILLE', 'HEIGHT',
                                'REPUBLIQUE', 'CAMEROUN', 'REPUBLIC', 'CAMEROON',
                                'CARTE', 'NATIONALE', 'IDENTITE', 'IDENTITY', 'CARD',
                                'PROFESSION', 'OCCUPATION', 'SIGNATURE'])
    
    def log(self, message: str, level: str = "INFO"):
        """Affiche un message de log si le mode debug est activé."""
        if self.debug:
            print(f"[{level}] {message}")
    
    def assess_quality(self, ocr_data: Dict) -> Tuple[bool, float]:
        """
        Évalue la qualité globale des données OCR.
        
        Returns:
            (peut_continuer, score_qualite)
        """
        self.log("=== ÉVALUATION DE LA QUALITÉ ===")
        
        scores = ocr_data.get('rec_scores', [])
        texts = ocr_data.get('rec_texts', [])
        
        if not scores or not texts:
            self.log("Pas de données OCR", "ERROR")
            return False, 0.0
        
        # Filtrer les scores valides (> 0)
        valid_scores = [s for s in scores if s > 0]
        
        if not valid_scores:
            self.log("Aucun score valide", "ERROR")
            return False, 0.0
        
        # Calculer le score moyen
        avg_score = sum(valid_scores) / len(valid_scores)
        
        # Compter les éléments de bonne qualité
        good_quality = sum(1 for s in scores if s > 0.7)
        
        self.log(f"Score moyen: {avg_score:.2f}")
        self.log(f"Éléments de bonne qualité (>0.7): {good_quality}/{len(scores)}")
        self.log(f"Éléments valides: {len(valid_scores)}")
        
        # Au moins 8 éléments détectés et score moyen acceptable
        can_proceed = (len(valid_scores) >= 8 and 
                      avg_score >= self.quality_threshold and
                      good_quality >= 5)
        
        self.log(f"Peut continuer: {can_proceed}")
        
        return can_proceed, avg_score
    
    def preprocess(self, texts: List[str], scores: List[float], 
                  polygons: List) -> List[Tuple[str, float, List]]:
        """
        Prétraite les données OCR en filtrant le bruit.
        
        Returns:
            Liste de tuples (texte, score, polygone) nettoyés
        """
        self.log("=== PRÉPROCESSING ===")
        processed = []
        
        for i, (text, score, polygon) in enumerate(zip(texts, scores, polygons)):
            original_text = text
            
            # Filtrer les scores trop faibles
            if score < 0.3:
                self.log(f"  Filtré (score faible): '{text}' (score={score:.2f})", "DEBUG")
                continue
            
            # Filtrer les textes vides
            text = text.strip()
            if not text:
                self.log(f"  Filtré (texte vide): index {i}", "DEBUG")
                continue
            
            # Filtrer les caractères non-latins isolés
            if len(text) <= 2:
                if any(ord(c) > 127 for c in text):
                    self.log(f"  Filtré (caractère non-latin): '{text}'", "DEBUG")
                    continue
            
            processed.append((text, score, polygon))
            self.log(f"  Gardé: '{text}' (score={score:.2f})")
        
        self.log(f"Éléments après préprocessing: {len(processed)}/{len(texts)}")
        
        return processed
    
    def similarity_score(self, str1: str, str2: str) -> float:
        """
        Calcule le score de similarité entre deux chaînes (Jaro-Winkler approximé).
        """
        # Normalisation
        s1 = str1.upper().strip()
        s2 = str2.upper().strip()
        
        # SequenceMatcher donne un bon compromis
        base_score = SequenceMatcher(None, s1, s2).ratio()
        
        # Bonus si les premiers caractères correspondent (Jaro-Winkler like)
        prefix_match = 0
        for i in range(min(4, len(s1), len(s2))):
            if s1[i] == s2[i]:
                prefix_match += 1
            else:
                break
        
        # Ajuster le score avec le bonus de préfixe
        final_score = base_score + (prefix_match * 0.1 * (1 - base_score))
        
        return min(final_score, 1.0)
    
    def extract_fixed_format_fields(self, data: List[Tuple[str, float, List]]) -> Dict:
        """
        Extrait les champs à format fixe (date, sexe, taille).
        
        Returns:
            Dictionnaire avec les champs extraits et les indices à retirer
        """
        self.log("=== EXTRACTION DES CHAMPS À FORMAT FIXE ===")
        
        results = {
            'date_naissance': None,
            'sexe': None,
            'taille': None
        }
        indices_to_remove = []
        
        # Patterns de détection
        date_pattern = re.compile(r'^\d{1,2}[./]\d{1,2}[./]\d{4}$')
        taille_pattern = re.compile(r'^[12][,.]?\d{2}$')
        
        for idx, (text, score, polygon) in enumerate(data):
            # Date de naissance
            if not results['date_naissance'] and date_pattern.match(text):
                self.log(f"  Date trouvée: '{text}' à l'index {idx}")
                results['date_naissance'] = text
                indices_to_remove.append(idx)
                continue
            
            # Sexe
            if not results['sexe'] and text in ['M', 'F']:
                self.log(f"  Sexe trouvé: '{text}' à l'index {idx}")
                results['sexe'] = text
                indices_to_remove.append(idx)
                continue
            
            # Taille
            if not results['taille'] and taille_pattern.match(text):
                # Normaliser le format
                taille = text.replace('.', ',')
                if ',' not in taille and len(taille) == 3:
                    taille = taille[0] + ',' + taille[1:]
                self.log(f"  Taille trouvée: '{text}' -> '{taille}' à l'index {idx}")
                results['taille'] = taille
                indices_to_remove.append(idx)
                continue
        
        self.log(f"Champs fixes extraits: Date={results['date_naissance']}, Sexe={results['sexe']}, Taille={results['taille']}")
        self.log(f"Indices à retirer: {indices_to_remove}")
        
        # Retirer les éléments extraits de la liste
        results['indices_removed'] = indices_to_remove
        
        return results
    
    def detect_anchors(self, data: List[Tuple[str, float, List]]) -> Dict[str, List[Tuple[int, str, float]]]:
        """
        Détecte les ancres avec similarité floue.
        
        Returns:
            Dictionnaire {type_champ: [(indice, texte_trouvé, score_similarité)]}
        """
        self.log("=== DÉTECTION DES ANCRES ===")
        
        detected = {field: [] for field in self.anchors}
        
        for idx, (text, score, _) in enumerate(data):
            text_upper = text.upper()
            
            # Vérifier contre chaque type d'ancre
            for field, anchor_list in self.anchors.items():
                for anchor in anchor_list:
                    sim_score = self.similarity_score(text_upper, anchor)
                    
                    if sim_score >= self.similarity_threshold:
                        self.log(f"  Ancre détectée pour '{field}': '{text}' ~ '{anchor}' (similarité={sim_score:.2f})")
                        detected[field].append((idx, text, sim_score))
                        break
                    elif sim_score > 0.7:  # Log les correspondances proches
                        self.log(f"    Correspondance proche pour '{field}': '{text}' ~ '{anchor}' (similarité={sim_score:.2f})", "DEBUG")
        
        # Résumé des ancres détectées
        for field, anchors in detected.items():
            if anchors:
                self.log(f"  {field}: {len(anchors)} ancre(s) trouvée(s)")
            else:
                self.log(f"  {field}: aucune ancre trouvée", "WARNING")
        
        return detected
    
    def is_likely_label(self, text: str) -> bool:
        """
        Vérifie si un texte ressemble à un label plutôt qu'à une valeur.
        """
        text_upper = text.upper()
        
        # Vérifier si le texte contient un slash (caractéristique des labels bilingues)
        if '/' in text and any(word in text_upper for word in ['NOM', 'SURNAME', 'PRENOM', 'GIVEN', 'DATE', 'LIEU', 'PLACE', 'SEXE', 'SEX', 'TAILLE', 'HEIGHT']):
            self.log(f"    '{text}' identifié comme label (format bilingue avec /)", "DEBUG")
            return True
        
        # Vérifier la similarité avec tous les labels connus
        for label in self.all_labels:
            sim = self.similarity_score(text_upper, label)
            if sim >= 0.75:
                self.log(f"    '{text}' identifié comme label (similaire à '{label}', score={sim:.2f})", "DEBUG")
                return True
        
        # Vérifier les patterns de labels composés
        label_words = ['CARTE', 'NATIONALE', 'REPUBLIQUE', 'DATE', 'LIEU', 
                      'PLACE', 'BIRTH', 'NAISSANCE', 'IDENTITY', 'CARD',
                      'REPUBLIC', 'CAMEROON', 'CAMEROUN', 'OF', 'DE', 'DU',
                      'PRENOMS', 'PRENOM', 'GIVEN', 'NAMES', 'NAME', 'NOM', 'SURNAME']
        
        words = text_upper.split()
        if len(words) > 1:
            matches = sum(1 for word in words if word in label_words)
            if matches >= len(words) / 2:
                self.log(f"    '{text}' identifié comme label (mots clés: {matches}/{len(words)})", "DEBUG")
                return True
        
        # Vérifier si c'est exactement un mot clé
        if text_upper in label_words:
            self.log(f"    '{text}' identifié comme label (mot clé exact)", "DEBUG")
            return True
        
        return False
    
    def extract_by_proximity(self, data: List[Tuple[str, float, List]], 
                            anchor_idx: int, field_name: str) -> Optional[str]:
        """
        Extrait la valeur la plus proche d'une ancre.
        """
        self.log(f"  Recherche de valeur pour '{field_name}' près de l'ancre à l'index {anchor_idx}")
        
        if anchor_idx >= len(data):
            return None
        
        anchor_text = data[anchor_idx][0]
        anchor_polygon = data[anchor_idx][2]
        anchor_center = self.calculate_center(anchor_polygon)
        
        self.log(f"    Ancre: '{anchor_text}' au centre {anchor_center}")
        
        candidates = []
        
        for idx, (text, score, polygon) in enumerate(data):
            if idx == anchor_idx:
                continue
            
            # Ignorer les labels
            if self.is_likely_label(text):
                self.log(f"    Ignoré (label): '{text}'", "DEBUG")
                continue
            
            value_center = self.calculate_center(polygon)
            
            # Calculer la distance
            distance = ((value_center[0] - anchor_center[0])**2 + 
                       (value_center[1] - anchor_center[1])**2) ** 0.5
            
            # Vérifier la position relative (à droite ou en dessous)
            is_right = value_center[0] > anchor_center[0]
            is_below = value_center[1] > anchor_center[1]
            
            if is_right or is_below:
                # Prioriser les éléments proches
                proximity_score = 1 / (1 + distance/100)
                candidates.append({
                    'text': text,
                    'score': score * proximity_score,
                    'distance': distance,
                    'position': 'droite' if is_right else 'dessous'
                })
                self.log(f"    Candidat: '{text}' ({candidates[-1]['position']}, distance={distance:.0f}, score={candidates[-1]['score']:.2f})")
        
        if candidates:
            # Prendre le meilleur candidat
            best = max(candidates, key=lambda x: x['score'])
            self.log(f"  Meilleur candidat pour '{field_name}': '{best['text']}'")
            return best['text']
        
        self.log(f"  Aucun candidat trouvé pour '{field_name}'", "WARNING")
        return None
    
    def calculate_center(self, polygon: List[List[int]]) -> Tuple[float, float]:
        """Calcule le centre d'un polygone."""
        x_coords = [p[0] for p in polygon]
        y_coords = [p[1] for p in polygon]
        return (sum(x_coords) / len(x_coords), sum(y_coords) / len(y_coords))
    
    def extract_remaining_fields(self, data: List[Tuple[str, float, List]], 
                                anchors: Dict[str, List[Tuple[int, str, float]]]) -> Dict:
        """
        Extrait nom, prénom et lieu de naissance.
        """
        self.log("=== EXTRACTION DES CHAMPS RESTANTS ===")
        
        results = {
            'nom': None,
            'prenom': None,
            'lieu_naissance': None,
            'profession': None,
        }
        
        used_indices = set()
        used_values = set()  # Pour éviter les doublons
        
        # Si on a des ancres, les utiliser
        for field in ['nom', 'prenom', 'lieu_naissance', 'profession']:
            if anchors[field]:
                # Prendre l'ancre avec le meilleur score de similarité
                best_anchor = max(anchors[field], key=lambda x: x[2])
                anchor_idx = best_anchor[0]
                
                self.log(f"Extraction par ancre pour '{field}':")
                value = self.extract_by_proximity(data, anchor_idx, field)
                
                if value and not self.is_likely_label(value) and value not in used_values:
                    results[field] = value
                    used_values.add(value)
                    # Marquer l'indice comme utilisé
                    for idx, (text, _, _) in enumerate(data):
                        if text == value:
                            used_indices.add(idx)
                            self.log(f"  '{field}' = '{value}' (extrait par ancre)")
                            break
                else:
                    self.log(f"  Échec de l'extraction par ancre pour '{field}'", "WARNING")
        
        # Pour les champs manquants, utiliser l'analyse positionnelle
        self.log("Extraction positionnelle pour les champs manquants:")
        
        remaining_texts = []
        for idx, (text, score, polygon) in enumerate(data):
            if idx not in used_indices and not self.is_likely_label(text) and text not in used_values:
                y_pos = self.calculate_center(polygon)[1]
                remaining_texts.append({
                    'text': text,
                    'score': score,
                    'y_position': y_pos,
                    'index': idx
                })
                self.log(f"  Texte restant: '{text}' (y={y_pos:.0f}, score={score:.2f})")
        
        # Trier par position verticale
        remaining_texts.sort(key=lambda x: x['y_position'])
        
        # Assigner les champs manquants par position
        if not results['nom'] and remaining_texts:
            results['nom'] = remaining_texts[0]['text']
            used_values.add(remaining_texts[0]['text'])
            self.log(f"  'nom' = '{results['nom']}' (position: premier élément)")
            remaining_texts.pop(0)
        
        if not results['prenom'] and remaining_texts:
            results['prenom'] = remaining_texts[0]['text']
            used_values.add(remaining_texts[0]['text'])
            self.log(f"  'prenom' = '{results['prenom']}' (position: deuxième élément)")
            remaining_texts.pop(0)
        
        if not results['lieu_naissance'] and remaining_texts:
            # Le lieu est souvent après le nom/prénom
            results['lieu_naissance'] = remaining_texts[0]['text']
            self.log(f"  'lieu_naissance' = '{results['lieu_naissance']}' (position: troisième élément)")
        
        return results
    
    def extract(self, ocr_data: Dict) -> Dict[str, any]:
        """
        Méthode principale d'extraction.
        
        Returns:
            Dictionnaire avec les champs extraits et les métadonnées
        """
        self.log("="*50)
        self.log("DÉBUT DE L'EXTRACTION CNI")
        self.log("="*50)
        
        # 1. Évaluer la qualité
        can_proceed, quality_score = self.assess_quality(ocr_data)
        
        if not can_proceed:
            return {
                'success': False,
                'quality_score': quality_score,
                'message': 'Qualité OCR insuffisante',
                'data': {}
            }
        
        # 2. Prétraitement
        texts = ocr_data.get('rec_texts', [])
        scores = ocr_data.get('rec_scores', [])
        polygons = ocr_data.get('rec_polys', [])
        
        processed_data = self.preprocess(texts, scores, polygons)
        
        # 3. Extraction des champs à format fixe
        fixed_fields = self.extract_fixed_format_fields(processed_data)
        
        # Retirer les éléments extraits
        remaining_data = [
            item for idx, item in enumerate(processed_data) 
            if idx not in fixed_fields['indices_removed']
        ]
        
        self.log(f"\nÉléments restants après extraction des champs fixes: {len(remaining_data)}")
        for text, score, _ in remaining_data:
            self.log(f"  - '{text}' (score={score:.2f})")
        
        # 4. Détecter les ancres
        anchors = self.detect_anchors(remaining_data)
        
        # 5. Extraire les champs restants
        remaining_fields = self.extract_remaining_fields(remaining_data, anchors)
        
        # 6. Consolider les résultats
        extracted_data = {
            'nom': remaining_fields['nom'],
            'prenom': remaining_fields['prenom'],
            'date_naissance': fixed_fields['date_naissance'],
            'lieu_naissance': remaining_fields['lieu_naissance'],
            'sexe': fixed_fields['sexe'],
            'taille': fixed_fields['taille'],
            'profession': remaining_fields['profession']
        }
        
        # Calculer le score de confiance
        filled_fields = sum(1 for v in extracted_data.values() if v is not None)
        confidence = filled_fields / 6.0
        
        self.log("\n=== RÉSULTATS FINAUX ===")
        for field, value in extracted_data.items():
            status = "[SUCCESS]" if value else "[FAILURE]"
            self.log(f"  {status} {field}: {value}")
        self.log(f"Confiance: {confidence:.0%}")
        
        return {
            'success': True,
            'quality_score': quality_score,
            'confidence': confidence,
            'anchors_detected': {k: len(v) > 0 for k, v in anchors.items()},
            'data': extracted_data
        }

# Exemple d'utilisation
if __name__ == "__main__":
    import json
    
    # Charger les données OCR
    with open("output/images/old/resizedold_front_02_res.json", 'r') as f:
        ocr_data = json.load(f)
    
    # Créer l'extracteur avec debug activé
    extractor = CNIExtractor18F(debug=True)
    
    # Extraire les informations
    result = extractor.extract(ocr_data)
    
    print("\n" + "="*50)
    print("RÉSUMÉ DE L'EXTRACTION")
    print("="*50)
    
    if result['success']:
        print(f"Extraction réussie (confiance: {result['confidence']:.0%})")
        print(f"Qualité OCR: {result['quality_score']:.2f}")
        print(f"Ancres détectées: {result['anchors_detected']}")
        print("\nDonnées extraites:")
        for field, value in result['data'].items():
            print(f"  {field}: {value}")
    else:
        print(f"Extraction échouée: {result['message']}")
        print(f"Qualité OCR: {result['quality_score']:.2f}")

[INFO] ==================================================
[INFO] DÉBUT DE L'EXTRACTION CNI
[INFO] ==================================================
[INFO] === ÉVALUATION DE LA QUALITÉ ===
[INFO] Score moyen: 0.92
[INFO] Éléments de bonne qualité (>0.7): 17/20
[INFO] Éléments valides: 18
[INFO] Peut continuer: True
[INFO] === PRÉPROCESSING ===
[DEBUG]   Filtré (score faible): '1' (score=0.07)
[INFO]   Gardé: 'REPUBLIQUE DU CAMEROUN' (score=0.97)
[INFO]   Gardé: 'REPUBLIC OF CAMEROON' (score=0.96)
[INFO]   Gardé: 'NOM/SURNAME' (score=0.99)
[INFO]   Gardé: 'AKONDACK AITAOY' (score=0.98)
[INFO]   Gardé: 'PRENOMS/GIVEN NAMES' (score=0.98)
[INFO]   Gardé: 'YVETTE MARIAN' (score=0.98)
[DEBUG]   Filtré (score faible): '' (score=0.00)
[INFO]   Gardé: 'DATE DE NAISSANCE/DATE OF BIRTH' (score=0.95)
[INFO]   Gardé: '19.04.1999' (score=0.99)
[INFO]   Gardé: 'LIEU DE NAISSANCE/PLACE OF BIRTH' (score=0.93)
[INFO]   Gardé: 'BAFIA' (score=0.99)
[DEBUG]   Filtré (score faible): '' (score=0.00)
[INFO]  

In [43]:
import re
from typing import Dict, List, Tuple, Optional
from difflib import SequenceMatcher


class CNIExtractor18B:
    """
    Extracteur pour le verso de CNI camerounaise utilisant une approche par élimination
    avec détection floue des ancres.
    """

    def __init__(self, quality_threshold: float = 0.5, similarity_threshold: float = 0.70, debug: bool = True):
        """
        Initialise l'extracteur pour le verso.

        Args:
            quality_threshold: Seuil minimal de qualité OCR pour procéder
            similarity_threshold: Seuil de similarité pour la détection des ancres
            debug: Active les logs de débogage
        """
        self.quality_threshold = quality_threshold
        self.similarity_threshold = similarity_threshold
        self.debug = debug

        # Ancres possibles pour chaque champ du verso
        self.anchors = {
            'pere': ['PERE', 'FATHER', 'PERE/FATHER'],
            'mere': ['MERE', 'MOTHER', 'MERE/MOTHER'],
            'date_delivrance': ['DATE DE DELIVRANCE', 'DATE OF ISSUE', 
                               'DATE DE DELIVRANCEI', 'DATEOFISSUE'],
            'date_expiration': ['DATE D\'EXPIRATION', 'DATE OF EXPIRY',
                              'DATED\'EXPIRATION', 'DATEOF EXPIRY'],
            'adresse': ['ADRESSE', 'ADDRESS', 'ADRESSE/ADDRESS'],
            'poste_identification': ['POSTE D\'IDENTIFICATION', 'IDENTIFICATION POST',
                                    'POSTE DIDENTIFICATION', 'POSTE DIDENTIFICATIONA'],
            'identifiant_unique': ['IDENTIFIANT UNIQUE', 'UNIQUE IDENTIFIER',
                                  'IDENTIFIANTUNIQUE', 'UNIQUEIDENTIFIER',
                                  'IDENTIFIANTUNIQUEI', 'UNIQUEIDENTIFIERI'],
            'autorite': ['AUTORITE', 'AUTHORITY', 'AUTORITE/AUTHORITY']
        }

        # Tous les labels possibles (pour filtrage)
        self.all_labels = set()
        for labels in self.anchors.values():
            self.all_labels.update(labels)
        self.all_labels.update(['S.P.', 'S.M.', 'S.P./S.M.', 
                                'CAMEROUN', 'CAMEROON'])

    def log(self, message: str, level: str = "INFO"):
        """Affiche un message de log si le mode debug est activé."""
        if self.debug:
            print(f"[{level}] {message}")

    def assess_quality(self, ocr_data: Dict) -> Tuple[bool, float]:
        """
        Évalue la qualité globale des données OCR.

        Returns:
            (peut_continuer, score_qualite)
        """
        self.log("=== ÉVALUATION DE LA QUALITÉ ===")

        scores = ocr_data.get('rec_scores', [])
        texts = ocr_data.get('rec_texts', [])

        if not scores or not texts:
            self.log("Pas de données OCR", "ERROR")
            return False, 0.0

        # Filtrer les scores valides (> 0)
        valid_scores = [s for s in scores if s > 0]

        if not valid_scores:
            self.log("Aucun score valide", "ERROR")
            return False, 0.0

        # Calculer le score moyen
        avg_score = sum(valid_scores) / len(valid_scores)

        # Compter les éléments de bonne qualité
        good_quality = sum(1 for s in scores if s > 0.7)

        self.log(f"Score moyen: {avg_score:.2f}")
        self.log(f"Éléments de bonne qualité (>0.7): {good_quality}/{len(scores)}")
        self.log(f"Éléments valides: {len(valid_scores)}")

        # Au moins 5 éléments détectés pour le verso
        can_proceed = (len(valid_scores) >= 5 and 
                      avg_score >= self.quality_threshold and
                      good_quality >= 3)

        self.log(f"Peut continuer: {can_proceed}")

        return can_proceed, avg_score

    def preprocess(self, texts: List[str], scores: List[float],
                  polygons: List) -> List[Tuple[str, float, List]]:
        """
        Prétraite les données OCR en filtrant le bruit.
        Pour le moment, traitement minimal comme demandé.
        """
        self.log("=== PRÉPROCESSING (MINIMAL) ===")
        processed = []

        for i, (text, score, polygon) in enumerate(zip(texts, scores, polygons)):
            # Filtrer les scores très faibles
            if score < 0.3:
                self.log(f"  Filtré (score faible): '{text}' (score={score:.2f})", "DEBUG")
                continue

            # Filtrer les textes vides
            text = text.strip()
            if not text:
                self.log(f"  Filtré (texte vide): index {i}", "DEBUG")
                continue

            # Filtrer les caractères non-latins isolés (鸡, 川)
            if len(text) <= 2:
                if any(ord(c) > 127 for c in text):
                    self.log(f"  Filtré (caractère non-latin): '{text}'", "DEBUG")
                    continue

            processed.append((text, score, polygon))
            self.log(f"  Gardé: '{text}' (score={score:.2f})")

        self.log(f"Éléments après préprocessing: {len(processed)}/{len(texts)}")

        return processed

    def similarity_score(self, str1: str, str2: str) -> float:
        """
        Calcule le score de similarité entre deux chaînes.
        """
        s1 = str1.upper().strip()
        s2 = str2.upper().strip()

        base_score = SequenceMatcher(None, s1, s2).ratio()

        # Bonus pour préfixe identique
        prefix_match = 0
        for i in range(min(4, len(s1), len(s2))):
            if s1[i] == s2[i]:
                prefix_match += 1
            else:
                break

        final_score = base_score + (prefix_match * 0.1 * (1 - base_score))

        return min(final_score, 1.0)

    def extract_fixed_format_fields(self, data: List[Tuple[str, float, List]]) -> Dict:
        """
        Extrait les champs à format fixe (dates, numéros).
        """
        self.log("=== EXTRACTION DES CHAMPS À FORMAT FIXE ===")

        results = {
            'date_delivrance': None,
            'date_expiration': None,
            'identifiant_unique': None,
            'numero_cni': None,
            'poste_code': None
        }
        indices_to_remove = []

        # Patterns de détection
        date_pattern = re.compile(r'^\d{1,2}[./]\d{1,2}[./]\d{4}$')
        id_unique_pattern = re.compile(r'^\d{15,20}$')  # Identifiant unique long
        numero_pattern = re.compile(r'^\d{9}$')  # Numéro CNI (9 chiffres)
        poste_pattern = re.compile(r'^[A-Z]{2}\d{2}$')  # Code poste (ex: LT02)

        dates_found = []
        
        for idx, (text, score, polygon) in enumerate(data):
            # Dates (on en attend 2 : délivrance et expiration)
            if date_pattern.match(text):
                dates_found.append((text, idx))
                self.log(f"  Date trouvée: '{text}' à l'index {idx}")
                indices_to_remove.append(idx)
                continue
            
            # Identifiant unique
            if not results['identifiant_unique'] and id_unique_pattern.match(text):
                self.log(f"  Identifiant unique trouvé: '{text}' à l'index {idx}")
                results['identifiant_unique'] = text
                indices_to_remove.append(idx)
                continue
            
            # Numéro CNI
            if not results['numero_cni'] and numero_pattern.match(text):
                self.log(f"  Numéro CNI trouvé: '{text}' à l'index {idx}")
                results['numero_cni'] = text
                indices_to_remove.append(idx)
                continue
            
            # Code poste
            if not results['poste_code'] and poste_pattern.match(text):
                self.log(f"  Code poste trouvé: '{text}' à l'index {idx}")
                results['poste_code'] = text
                indices_to_remove.append(idx)
                continue

        # Assigner les dates (première = délivrance, deuxième = expiration)
        if len(dates_found) >= 1:
            results['date_delivrance'] = dates_found[0][0]
        if len(dates_found) >= 2:
            results['date_expiration'] = dates_found[1][0]

        self.log(f"Champs fixes extraits: {len([v for v in results.values() if v])} sur 5")
        self.log(f"Indices à retirer: {indices_to_remove}")

        results['indices_removed'] = indices_to_remove

        return results

    def detect_anchors(self, data: List[Tuple[str, float, List]]) -> Dict[str, List[Tuple[int, str, float]]]:
        """
        Détecte les ancres avec similarité floue.
        """
        self.log("=== DÉTECTION DES ANCRES ===")

        detected = {field: [] for field in self.anchors}

        for idx, (text, score, _) in enumerate(data):
            text_upper = text.upper()

            for field, anchor_list in self.anchors.items():
                for anchor in anchor_list:
                    sim_score = self.similarity_score(text_upper, anchor)

                    if sim_score >= self.similarity_threshold:
                        self.log(f"  Ancre détectée pour '{field}': '{text}' ~ '{anchor}' (similarité={sim_score:.2f})")
                        detected[field].append((idx, text, sim_score))
                        break

        # Résumé des ancres détectées
        for field, anchors in detected.items():
            if anchors:
                self.log(f"  {field}: {len(anchors)} ancre(s) trouvée(s)")
            else:
                self.log(f"  {field}: aucune ancre trouvée", "WARNING")

        return detected

    def is_likely_label(self, text: str) -> bool:
        """
        Vérifie si un texte ressemble à un label plutôt qu'à une valeur.
        """
        text_upper = text.upper()

        # Vérifier si c'est un format bilingue avec /
        if '/' in text and any(word in text_upper for word in ['PERE', 'FATHER', 'MERE', 'MOTHER', 
                                                                'DATE', 'ADRESSE', 'ADDRESS']):
            self.log(f"    '{text}' identifié comme label (format bilingue avec /)", "DEBUG")
            return True

        # Vérifier la similarité avec tous les labels connus
        for label in self.all_labels:
            sim = self.similarity_score(text_upper, label)
            if sim >= 0.75:
                self.log(f"    '{text}' identifié comme label (similaire à '{label}', score={sim:.2f})", "DEBUG")
                return True

        # Mots-clés spécifiques au verso
        label_words = ['PERE', 'FATHER', 'MERE', 'MOTHER', 'DATE', 'DELIVRANCE', 
                      'ISSUE', 'EXPIRATION', 'EXPIRY', 'ADRESSE', 'ADDRESS',
                      'POSTE', 'IDENTIFICATION', 'POST', 'IDENTIFIANT', 'UNIQUE',
                      'IDENTIFIER', 'AUTORITE', 'AUTHORITY', 'CAMEROUN', 'CAMEROON']

        words = text_upper.split()
        if len(words) > 1:
            matches = sum(1 for word in words if word in label_words)
            if matches >= len(words) / 2:
                self.log(f"    '{text}' identifié comme label (mots clés: {matches}/{len(words)})", "DEBUG")
                return True

        return False

    def extract_by_proximity(self, data: List[Tuple[str, float, List]],
                            anchor_idx: int, field_name: str) -> Optional[str]:
        """
        Extrait la valeur la plus proche d'une ancre.
        """
        self.log(f"  Recherche de valeur pour '{field_name}' près de l'ancre à l'index {anchor_idx}")

        if anchor_idx >= len(data):
            return None

        anchor_text = data[anchor_idx][0]
        anchor_polygon = data[anchor_idx][2]
        anchor_center = self.calculate_center(anchor_polygon)

        candidates = []

        for idx, (text, score, polygon) in enumerate(data):
            if idx == anchor_idx:
                continue

            if self.is_likely_label(text):
                self.log(f"    Ignoré (label): '{text}'", "DEBUG")
                continue

            value_center = self.calculate_center(polygon)

            # Distance
            distance = ((value_center[0] - anchor_center[0])**2 + 
                       (value_center[1] - anchor_center[1])**2) ** 0.5

            # Position relative
            is_right = value_center[0] > anchor_center[0]
            is_below = value_center[1] > anchor_center[1]

            if is_right or is_below:
                proximity_score = 1 / (1 + distance/100)
                candidates.append({
                    'text': text,
                    'score': score * proximity_score,
                    'distance': distance
                })
                self.log(f"    Candidat: '{text}' (distance={distance:.0f}, score={score * proximity_score:.2f})")

        if candidates:
            best = max(candidates, key=lambda x: x['score'])
            self.log(f"  Meilleur candidat pour '{field_name}': '{best['text']}'")
            return best['text']

        return None

    def calculate_center(self, polygon: List[List[int]]) -> Tuple[float, float]:
        """Calcule le centre d'un polygone."""
        x_coords = [p[0] for p in polygon]
        y_coords = [p[1] for p in polygon]
        return (sum(x_coords) / len(x_coords), sum(y_coords) / len(y_coords))

    def extract_remaining_fields(self, data: List[Tuple[str, float, List]], 
                                anchors: Dict[str, List[Tuple[int, str, float]]],
                                fixed_fields: Dict) -> Dict:
        """
        Extrait les champs restants (père, mère, adresse, autorité).
        """
        self.log("=== EXTRACTION DES CHAMPS RESTANTS ===")

        results = {
            'pere': None,
            'mere': None,
            'adresse': None,
            'autorite': None,
            'poste_identification': fixed_fields.get('poste_code')  # Déjà extrait
        }

        used_indices = set()
        used_values = set()

        # Extraction par ancres
        for field in ['pere', 'mere', 'adresse', 'autorite']:
            if anchors.get(field, []):
                best_anchor = max(anchors[field], key=lambda x: x[2])
                anchor_idx = best_anchor[0]

                self.log(f"Extraction par ancre pour '{field}':")
                value = self.extract_by_proximity(data, anchor_idx, field)

                if value and not self.is_likely_label(value) and value not in used_values:
                    results[field] = value
                    used_values.add(value)
                    self.log(f"  '{field}' = '{value}' (extrait par ancre)")

        # Pour l'autorité, chercher aussi les noms avec format spécifique
        if not results['autorite']:
            for text, score, _ in data:
                # Pattern pour nom complet (ex: Martin MBARGA NGUELE)
                if score > 0.9 and len(text.split()) >= 2:
                    words = text.split()
                    # Vérifier si c'est un nom propre (mots commençant par majuscule)
                    if all(w[0].isupper() for w in words if w):
                        if text not in used_values and not self.is_likely_label(text):
                            results['autorite'] = text
                            self.log(f"  'autorite' = '{text}' (détection pattern nom)")
                            break

        return results

    def extract(self, ocr_data: Dict) -> Dict[str, any]:
        """
        Méthode principale d'extraction pour le verso.
        """
        self.log("="*50)
        self.log("DÉBUT DE L'EXTRACTION CNI (VERSO)")
        self.log("="*50)

        # 1. Évaluer la qualité
        can_proceed, quality_score = self.assess_quality(ocr_data)

        if not can_proceed:
            return {
                'success': False,
                'quality_score': quality_score,
                'message': 'Qualité OCR insuffisante',
                'data': {}
            }

        # 2. Prétraitement
        texts = ocr_data.get('rec_texts', [])
        scores = ocr_data.get('rec_scores', [])
        polygons = ocr_data.get('rec_polys', [])

        processed_data = self.preprocess(texts, scores, polygons)

        # 3. Extraction des champs à format fixe
        fixed_fields = self.extract_fixed_format_fields(processed_data)

        # Retirer les éléments extraits
        remaining_data = [
            item for idx, item in enumerate(processed_data)
            if idx not in fixed_fields['indices_removed']
        ]

        self.log(f"\nÉléments restants après extraction des champs fixes: {len(remaining_data)}")

        # 4. Détecter les ancres
        anchors = self.detect_anchors(remaining_data)

        # 5. Extraire les champs restants
        remaining_fields = self.extract_remaining_fields(remaining_data, anchors, fixed_fields)

        # 6. Consolider les résultats
        extracted_data = {
            'pere': remaining_fields['pere'],
            'mere': remaining_fields['mere'],
            'date_delivrance': fixed_fields['date_delivrance'],
            'date_expiration': fixed_fields['date_expiration'],
            'adresse': remaining_fields['adresse'],
            'poste_identification': remaining_fields.get('poste_identification') or fixed_fields.get('poste_code'),
            'identifiant_unique': fixed_fields['identifiant_unique'],
            'autorite': remaining_fields['autorite'],
            'numero_cni': fixed_fields.get('numero_cni')
        }

        # Score de confiance
        filled_fields = sum(1 for v in extracted_data.values() if v is not None)
        total_fields = len(extracted_data)
        confidence = filled_fields / total_fields if total_fields > 0 else 0

        self.log("\n=== RÉSULTATS FINAUX ===")
        for field, value in extracted_data.items():
            status = "[SUCCESS]" if value else "[FAILURE]"
            self.log(f"  {status} {field}: {value}")
        self.log(f"Confiance: {confidence:.0%}")

        return {
            'success': True,
            'quality_score': quality_score,
            'confidence': confidence,
            'anchors_detected': {k: len(v) > 0 for k, v in anchors.items()},
            'data': extracted_data
        }


# Exemple d'utilisation
if __name__ == "__main__":
    import json
    
    # Charger les données OCR du verso
    with open('output/images/old/resizedold_back_02_res.json', 'r', encoding='utf-8') as f:
        ocr_data = json.load(f)
    
    # Créer l'extracteur pour le verso
    extractor = CNIExtractor18B(debug=True)
    
    # Extraire les informations
    result = extractor.extract(ocr_data)
    
    print("\n" + "="*50)
    print("RÉSUMÉ DE L'EXTRACTION (VERSO)")
    print("="*50)
    
    if result['success']:
        print(f"Extraction réussie (confiance: {result['confidence']:.0%})")
        print(f"Qualité OCR: {result['quality_score']:.2f}")
        print("\nDonnées extraites:")
        for field, value in result['data'].items():
            if value:
                print(f"  {field}: {value}")
    else:
        print(f"Extraction échouée: {result['message']}")

[INFO] ==================================================
[INFO] DÉBUT DE L'EXTRACTION CNI (VERSO)
[INFO] ==================================================
[INFO] === ÉVALUATION DE LA QUALITÉ ===
[INFO] Score moyen: 0.93
[INFO] Éléments de bonne qualité (>0.7): 26/29
[INFO] Éléments valides: 28
[INFO] Peut continuer: True
[INFO] === PRÉPROCESSING (MINIMAL) ===
[DEBUG]   Filtré (caractère non-latin): '鸡'
[DEBUG]   Filtré (score faible): '' (score=0.00)
[INFO]   Gardé: 'PERE/FATHER' (score=0.96)
[INFO]   Gardé: 'TSOYIIA DEEDI RENE' (score=0.91)
[INFO]   Gardé: '0' (score=0.98)
[INFO]   Gardé: 'MERE/MOTHER' (score=0.98)
[INFO]   Gardé: 'KAMANA ROSALIE' (score=0.98)
[INFO]   Gardé: 'S.P./S.M.' (score=0.95)
[INFO]   Gardé: 'AUTORITE/AUTHORITY' (score=0.99)
[INFO]   Gardé: 'DATE DE DELIVRANCEI' (score=0.93)
[INFO]   Gardé: 'POSTE DIDENTIFICATIONA' (score=0.94)
[INFO]   Gardé: 'DATE OFISSUE' (score=0.96)
[INFO]   Gardé: '860000' (score=1.00)
[INFO]   Gardé: 'IDENTIFICATION POST' (score=0.95)

In [48]:
import re
from typing import Dict, List, Tuple, Optional
from difflib import SequenceMatcher


class CNIExtractor2025F:
    """
    Extracteur pour le nouveau format de CNI camerounaise (2025) 
    utilisant une approche par élimination avec détection floue des ancres.
    """

    def __init__(self, quality_threshold: float = 0.5, similarity_threshold: float = 0.70, debug: bool = True):
        """
        Initialise l'extracteur pour le nouveau format.

        Args:
            quality_threshold: Seuil minimal de qualité OCR pour procéder
            similarity_threshold: Seuil de similarité pour la détection des ancres
            debug: Active les logs de débogage
        """
        self.quality_threshold = quality_threshold
        self.similarity_threshold = similarity_threshold
        self.debug = debug

        # Ancres possibles pour chaque champ
        self.anchors = {
            'nom': ['NOM', 'SURNAME', 'NOM/SURNAME'],
            'prenom': ['PRENOMS', 'PRENOM', 'GIVEN NAMES', 'GIVEN NAME', 
                      'PRENOMS/GIVEN NAMES', 'PRENOMS/GIVEN NAME'],
            'date_naissance': ['DATE DE NAISSANCE', 'DATE OF BIRTH', 
                             'DATE DENAISSANCE', 'DATEOF BIRTH',
                             'DATE DE NAISSANCE/DATE OF BIRTH'],
            'date_expiration': ['DATE D\'EXPIRATION', 'DATE OF EXPIRY',
                              'DATED\'EXPIRATION', 'DATEOF EXPIRY',
                              'DATE D\'EXPIRATION/DATE OF EXPIRY'],
            'sexe': ['SEXE', 'SEX', 'SEXE/SEX'],
            'signature': ['SIGNATURE', 'HOLDER\'S SIGNATURE', 
                         'SIGNATURE/HOLDER\'S SIGNATURE']
        }

        # Labels à ignorer (filigranes et textes de fond)
        self.ignore_words = {
            'TRAVAIL', 'PATRIE', 'WORK', 'FATHERLAND', 
            'CMR', 'CAMEROUN', 'CAMEROON',
            'REPUBLIQUE', 'REPUBLIC', 'DU', 'OF',
            'CARTE', 'NATIONALE', 'IDENTITE', 
            'NATIONAL', 'IDENTITY', 'CARD'
        }

        # Tous les labels possibles (pour filtrage)
        self.all_labels = set()
        for labels in self.anchors.values():
            self.all_labels.update(labels)
        self.all_labels.update(self.ignore_words)
        self.all_labels.update(['SIGNATURE', 'HOLDER\'S'])

    def log(self, message: str, level: str = "INFO"):
        """Affiche un message de log si le mode debug est activé."""
        if self.debug:
            print(f"[{level}] {message}")

    def assess_quality(self, ocr_data: Dict) -> Tuple[bool, float]:
        """
        Évalue la qualité globale des données OCR.

        Returns:
            (peut_continuer, score_qualite)
        """
        self.log("=== ÉVALUATION DE LA QUALITÉ ===")

        scores = ocr_data.get('rec_scores', [])
        texts = ocr_data.get('rec_texts', [])

        if not scores or not texts:
            self.log("Pas de données OCR", "ERROR")
            return False, 0.0

        # Filtrer les scores valides (> 0)
        valid_scores = [s for s in scores if s > 0]

        if not valid_scores:
            self.log("Aucun score valide", "ERROR")
            return False, 0.0

        # Calculer le score moyen
        avg_score = sum(valid_scores) / len(valid_scores)

        # Compter les éléments de bonne qualité
        good_quality = sum(1 for s in scores if s > 0.7)

        self.log(f"Score moyen: {avg_score:.2f}")
        self.log(f"Éléments de bonne qualité (>0.7): {good_quality}/{len(scores)}")
        self.log(f"Éléments valides: {len(valid_scores)}")

        # Au moins 6 éléments détectés pour le nouveau format
        can_proceed = (len(valid_scores) >= 6 and 
                      avg_score >= self.quality_threshold and
                      good_quality >= 4)

        self.log(f"Peut continuer: {can_proceed}")

        return can_proceed, avg_score

    def preprocess(self, texts: List[str], scores: List[float],
                  polygons: List) -> List[Tuple[str, float, List]]:
        """
        Prétraite les données OCR en filtrant le bruit et les filigranes.
        """
        self.log("=== PRÉPROCESSING ===")
        processed = []

        for i, (text, score, polygon) in enumerate(zip(texts, scores, polygons)):
            # Filtrer les scores très faibles
            if score < 0.3:
                self.log(f"  Filtré (score faible): '{text}' (score={score:.2f})", "DEBUG")
                continue

            # Filtrer les textes vides
            text = text.strip()
            if not text:
                self.log(f"  Filtré (texte vide): index {i}", "DEBUG")
                continue

            # Filtrer les caractères non-latins isolés (国, etc.)
            if len(text) <= 2:
                if any(ord(c) > 127 for c in text):
                    self.log(f"  Filtré (caractère non-latin): '{text}'", "DEBUG")
                    continue

            # Filtrer les mots isolés qui sont des filigranes connus
            if text.upper() in self.ignore_words:
                self.log(f"  Filtré (filigrane): '{text}'", "DEBUG")
                continue

            # Filtrer CMR et codes pays de 3 lettres
            if len(text) == 3 and text.isupper() and text.isalpha():
                self.log(f"  Filtré (code pays probable): '{text}'", "DEBUG")
                continue

            processed.append((text, score, polygon))
            self.log(f"  Gardé: '{text}' (score={score:.2f})")

        self.log(f"Éléments après préprocessing: {len(processed)}/{len(texts)}")

        return processed

    def similarity_score(self, str1: str, str2: str) -> float:
        """
        Calcule le score de similarité entre deux chaînes.
        """
        s1 = str1.upper().strip()
        s2 = str2.upper().strip()

        base_score = SequenceMatcher(None, s1, s2).ratio()

        # Bonus pour préfixe identique
        prefix_match = 0
        for i in range(min(4, len(s1), len(s2))):
            if s1[i] == s2[i]:
                prefix_match += 1
            else:
                break

        final_score = base_score + (prefix_match * 0.1 * (1 - base_score))

        return min(final_score, 1.0)

    def extract_fixed_format_fields(self, data: List[Tuple[str, float, List]]) -> Dict:
        """
        Extrait les champs à format fixe (numéro, dates, sexe).
        """
        self.log("=== EXTRACTION DES CHAMPS À FORMAT FIXE ===")

        results = {
            'numero_carte': None,
            'date_naissance': None,
            'date_expiration': None,
            'sexe': None
        }
        indices_to_remove = []

        # Patterns de détection
        date_pattern = re.compile(r'^\d{1,2}\.\d{1,2}\.\d{4}$')  # Format avec points
        numero_pattern = re.compile(r'^\d{9}$')  # Numéro de carte (9 chiffres)

        dates_found = []
        
        for idx, (text, score, polygon) in enumerate(data):
            # Numéro de carte (9 chiffres)
            if not results['numero_carte'] and numero_pattern.match(text):
                self.log(f"  Numéro de carte trouvé: '{text}' à l'index {idx}")
                results['numero_carte'] = text
                indices_to_remove.append(idx)
                continue
            
            # Dates (format avec points)
            if date_pattern.match(text):
                dates_found.append((text, idx, polygon))
                self.log(f"  Date trouvée: '{text}' à l'index {idx}")
                indices_to_remove.append(idx)
                continue
            
            # Sexe
            if not results['sexe'] and text in ['M', 'F']:
                self.log(f"  Sexe trouvé: '{text}' à l'index {idx}")
                results['sexe'] = text
                indices_to_remove.append(idx)
                continue

        # Distinguer les dates par leur année
        for date_text, idx, polygon in dates_found:
            year = int(date_text.split('.')[-1])
            
            # Date de naissance : année < 2010 généralement
            if not results['date_naissance'] and year < 2010:
                results['date_naissance'] = date_text
                self.log(f"  Date de naissance identifiée: {date_text} (année {year})")
            
            # Date d'expiration : année > 2020 généralement
            elif not results['date_expiration'] and year > 2020:
                results['date_expiration'] = date_text
                self.log(f"  Date d'expiration identifiée: {date_text} (année {year})")

        # Si on n'a pas pu distinguer par l'année, prendre par ordre
        if not results['date_naissance'] and dates_found:
            results['date_naissance'] = dates_found[0][0]
        if not results['date_expiration'] and len(dates_found) > 1:
            results['date_expiration'] = dates_found[1][0]

        self.log(f"Champs fixes extraits: {len([v for v in results.values() if v])} sur 4")
        self.log(f"Indices à retirer: {indices_to_remove}")

        results['indices_removed'] = indices_to_remove

        return results

    def detect_anchors(self, data: List[Tuple[str, float, List]]) -> Dict[str, List[Tuple[int, str, float]]]:
        """
        Détecte les ancres avec similarité floue.
        """
        self.log("=== DÉTECTION DES ANCRES ===")

        detected = {field: [] for field in self.anchors}

        for idx, (text, score, _) in enumerate(data):
            text_upper = text.upper()

            for field, anchor_list in self.anchors.items():
                for anchor in anchor_list:
                    sim_score = self.similarity_score(text_upper, anchor)

                    if sim_score >= self.similarity_threshold:
                        self.log(f"  Ancre détectée pour '{field}': '{text}' ~ '{anchor}' (similarité={sim_score:.2f})")
                        detected[field].append((idx, text, sim_score))
                        break

        # Résumé des ancres détectées
        for field, anchors in detected.items():
            if anchors:
                self.log(f"  {field}: {len(anchors)} ancre(s) trouvée(s)")
            else:
                self.log(f"  {field}: aucune ancre trouvée", "WARNING")

        return detected

    def is_likely_label(self, text: str) -> bool:
        """
        Vérifie si un texte ressemble à un label plutôt qu'à une valeur.
        """
        text_upper = text.upper()

        # Vérifier si c'est un mot isolé de filigrane
        if text_upper in self.ignore_words:
            self.log(f"    '{text}' identifié comme filigrane", "DEBUG")
            return True

        # Vérifier si c'est un format bilingue avec /
        if '/' in text and any(word in text_upper for word in ['NOM', 'SURNAME', 'PRENOM', 'GIVEN', 
                                                                'DATE', 'SEXE', 'SEX', 'SIGNATURE']):
            self.log(f"    '{text}' identifié comme label (format bilingue avec /)", "DEBUG")
            return True

        # Vérifier la similarité avec tous les labels connus
        for label in self.all_labels:
            sim = self.similarity_score(text_upper, label)
            if sim >= 0.75:
                self.log(f"    '{text}' identifié comme label (similaire à '{label}', score={sim:.2f})", "DEBUG")
                return True

        # Mots-clés spécifiques
        label_words = ['NOM', 'SURNAME', 'PRENOMS', 'PRENOM', 'GIVEN', 'NAMES', 'NAME',
                      'DATE', 'NAISSANCE', 'BIRTH', 'EXPIRATION', 'EXPIRY',
                      'SEXE', 'SEX', 'SIGNATURE', 'HOLDER\'S']

        words = text_upper.split()
        if len(words) > 1:
            matches = sum(1 for word in words if word in label_words)
            if matches >= len(words) / 2:
                self.log(f"    '{text}' identifié comme label (mots clés: {matches}/{len(words)})", "DEBUG")
                return True

        return False

    def extract_by_proximity(self, data: List[Tuple[str, float, List]],
                            anchor_idx: int, field_name: str) -> Optional[str]:
        """
        Extrait la valeur la plus proche d'une ancre.
        """
        self.log(f"  Recherche de valeur pour '{field_name}' près de l'ancre à l'index {anchor_idx}")

        if anchor_idx >= len(data):
            return None

        anchor_text = data[anchor_idx][0]
        anchor_polygon = data[anchor_idx][2]
        anchor_center = self.calculate_center(anchor_polygon)

        candidates = []

        for idx, (text, score, polygon) in enumerate(data):
            if idx == anchor_idx:
                continue

            if self.is_likely_label(text):
                self.log(f"    Ignoré (label): '{text}'", "DEBUG")
                continue

            value_center = self.calculate_center(polygon)

            # Distance
            distance = ((value_center[0] - anchor_center[0])**2 + 
                       (value_center[1] - anchor_center[1])**2) ** 0.5

            # Position relative
            is_right = value_center[0] > anchor_center[0]
            is_below = value_center[1] > anchor_center[1]

            if is_right or is_below:
                proximity_score = 1 / (1 + distance/100)
                candidates.append({
                    'text': text,
                    'score': score * proximity_score,
                    'distance': distance
                })
                self.log(f"    Candidat: '{text}' (distance={distance:.0f}, score={score * proximity_score:.2f})")

        if candidates:
            best = max(candidates, key=lambda x: x['score'])
            self.log(f"  Meilleur candidat pour '{field_name}': '{best['text']}'")
            return best['text']

        return None

    def calculate_center(self, polygon: List[List[int]]) -> Tuple[float, float]:
        """Calcule le centre d'un polygone."""
        x_coords = [p[0] for p in polygon]
        y_coords = [p[1] for p in polygon]
        return (sum(x_coords) / len(x_coords), sum(y_coords) / len(y_coords))

    def extract_remaining_fields(self, data: List[Tuple[str, float, List]], 
                                anchors: Dict[str, List[Tuple[int, str, float]]]) -> Dict:
        """
        Extrait les champs restants (nom, prénom).
        """
        self.log("=== EXTRACTION DES CHAMPS RESTANTS ===")

        results = {
            'nom': None,
            'prenom': None
        }

        used_values = set()

        # Extraction par ancres
        for field in ['nom', 'prenom']:
            if anchors.get(field, []):
                best_anchor = max(anchors[field], key=lambda x: x[2])
                anchor_idx = best_anchor[0]

                self.log(f"Extraction par ancre pour '{field}':")
                value = self.extract_by_proximity(data, anchor_idx, field)

                if value and not self.is_likely_label(value) and value not in used_values:
                    results[field] = value
                    used_values.add(value)
                    self.log(f"  '{field}' = '{value}' (extrait par ancre)")

        # Si les champs manquent, chercher les textes restants valides
        if not results['nom'] or not results['prenom']:
            remaining_texts = []
            for text, score, polygon in data:
                # Vérifier que c'est un nom valide (alphabétique, score élevé)
                if (text not in used_values and 
                    not self.is_likely_label(text) and
                    score > 0.9 and
                    text.isalpha() and
                    len(text) > 2):
                    
                    y_pos = self.calculate_center(polygon)[1]
                    remaining_texts.append({
                        'text': text,
                        'score': score,
                        'y_position': y_pos
                    })
                    self.log(f"  Texte candidat nom/prénom: '{text}' (y={y_pos:.0f}, score={score:.2f})")

            # Trier par position verticale
            remaining_texts.sort(key=lambda x: x['y_position'])

            # Assigner les champs manquants
            if not results['nom'] and remaining_texts:
                results['nom'] = remaining_texts[0]['text']
                self.log(f"  'nom' = '{results['nom']}' (position: premier élément)")
                remaining_texts.pop(0)

            if not results['prenom'] and remaining_texts:
                results['prenom'] = remaining_texts[0]['text']
                self.log(f"  'prenom' = '{results['prenom']}' (position: deuxième élément)")

        return results

    def extract(self, ocr_data: Dict) -> Dict[str, any]:
        """
        Méthode principale d'extraction pour le nouveau format.
        """
        self.log("="*50)
        self.log("DÉBUT DE L'EXTRACTION CNI (FORMAT 2025)")
        self.log("="*50)

        # 1. Évaluer la qualité
        can_proceed, quality_score = self.assess_quality(ocr_data)

        if not can_proceed:
            return {
                'success': False,
                'quality_score': quality_score,
                'message': 'Qualité OCR insuffisante',
                'data': {}
            }

        # 2. Prétraitement
        texts = ocr_data.get('rec_texts', [])
        scores = ocr_data.get('rec_scores', [])
        polygons = ocr_data.get('rec_polys', [])

        processed_data = self.preprocess(texts, scores, polygons)

        # 3. Extraction des champs à format fixe
        fixed_fields = self.extract_fixed_format_fields(processed_data)

        # Retirer les éléments extraits
        remaining_data = [
            item for idx, item in enumerate(processed_data)
            if idx not in fixed_fields['indices_removed']
        ]

        self.log(f"\nÉléments restants après extraction des champs fixes: {len(remaining_data)}")
        for text, score, _ in remaining_data:
            self.log(f"  - '{text}' (score={score:.2f})")

        # 4. Détecter les ancres
        anchors = self.detect_anchors(remaining_data)

        # 5. Extraire les champs restants
        remaining_fields = self.extract_remaining_fields(remaining_data, anchors)

        # 6. Consolider les résultats (gardant les mêmes clés pour la cohérence)
        extracted_data = {
            'numero_carte': fixed_fields['numero_carte'],
            'nom': remaining_fields['nom'],
            'prenom': remaining_fields['prenom'],
            'date_naissance': fixed_fields['date_naissance'],
            'date_expiration': fixed_fields['date_expiration'],
            'sexe': fixed_fields['sexe']
        }

        # Score de confiance
        filled_fields = sum(1 for v in extracted_data.values() if v is not None)
        total_fields = len(extracted_data)
        confidence = filled_fields / total_fields if total_fields > 0 else 0

        self.log("\n=== RÉSULTATS FINAUX ===")
        for field, value in extracted_data.items():
            status = "[SUCCESS]" if value else "[FAILURE]"
            self.log(f"  {status} {field}: {value}")
        self.log(f"Confiance: {confidence:.0%}")

        return {
            'success': True,
            'quality_score': quality_score,
            'confidence': confidence,
            'anchors_detected': {k: len(v) > 0 for k, v in anchors.items()},
            'data': extracted_data
        }


# Exemple d'utilisation
if __name__ == "__main__":
    import json
    
    # Charger les données OCR du nouveau format
    with open('output/images/new_front_02_res.json', 'r') as f:
        ocr_data = json.load(f)
    
    # Créer l'extracteur pour le nouveau format
    extractor = CNIExtractor2025F(debug=True)
    
    # Extraire les informations
    result = extractor.extract(ocr_data)
    
    print("\n" + "="*50)
    print("RÉSUMÉ DE L'EXTRACTION (FORMAT 2025)")
    print("="*50)
    
    if result['success']:
        print(f"Extraction réussie (confiance: {result['confidence']:.0%})")
        print(f"Qualité OCR: {result['quality_score']:.2f}")
        print("\nDonnées extraites:")
        for field, value in result['data'].items():
            if value:
                print(f"  {field}: {value}")
    else:
        print(f"Extraction échouée: {result['message']}")

[INFO] ==================================================
[INFO] DÉBUT DE L'EXTRACTION CNI (FORMAT 2025)
[INFO] ==================================================
[INFO] === ÉVALUATION DE LA QUALITÉ ===
[INFO] Score moyen: 0.93
[INFO] Éléments de bonne qualité (>0.7): 18/20
[INFO] Éléments valides: 19
[INFO] Peut continuer: True
[INFO] === PRÉPROCESSING ===
[DEBUG]   Filtré (score faible): 'å›½' (score=0.18)
[DEBUG]   Filtré (score faible): '' (score=0.00)
[INFO]   Gardé: '100121292' (score=0.95)
[DEBUG]   Filtré (filigrane): 'TRAVAIL'
[INFO]   Gardé: 'NOM/SURNAME' (score=0.99)
[DEBUG]   Filtré (filigrane): 'PATRIE'
[DEBUG]   Filtré (filigrane): 'WORK'
[DEBUG]   Filtré (filigrane): 'FATHERLAND'
[INFO]   Gardé: 'ZYANBA' (score=0.98)
[INFO]   Gardé: 'PRENOMS/GIVEN NAMES' (score=0.98)
[INFO]   Gardé: 'RAHAB' (score=0.99)
[INFO]   Gardé: 'SEXE/SEX' (score=0.98)
[INFO]   Gardé: 'DATE DE NAISSANCE/DATE OF BIRTH' (score=0.95)
[INFO]   Gardé: 'F' (score=0.99)
[INFO]   Gardé: '01.01.1974' (scor